# Modify JSON documents

With [JSON_MODIFY](https://docs.microsoft.com/en-us/sql/t-sql/functions/json-modify-transact-sql) it is possibile to manipulate JSON to add/remove/update elements.

In [3]:
declare @json varchar(max)
set @json = 
N'{
  "firstName": "John",
  "lastName": "Smith",
  "children": []
}';

select * from
( values  
	('Update existing value',           json_modify(@json, '$.firstName', 'Dave')),
	('Insert scalar value',             json_modify(@json, '$.isAlive', 'true')),
	('Insert array',                    json_modify(@json, '$.preferredColors', json_query('["Blue", "Black"]'))), -- please note json_query usage to avoid escaping
	('Append to array',                 json_modify(@json, 'append $.children', 'Annette')), 
	('Replace an array with a scalar',  json_modify(@json, '$.children', 'Annette')), 
	('Add an object',                   json_modify(@json, '$.phoneNumbers', json_query('{"type": "home","number": "212 555-1234"}'))), -- please note json_query usage to avoid escaping
	('Remove an object',                json_modify(@json, '$.firstName', null))
) t([action], result)
go



(7 rows affected)

Total execution time: 00:00:00.019

action,result
Update existing value,"{ ""firstName"": ""Dave"", ""lastName"": ""Smith"", ""children"": [] }"
Insert scalar value,"{ ""firstName"": ""John"", ""lastName"": ""Smith"", ""children"": [] ,""isAlive"":""true""}"
Insert array,"{ ""firstName"": ""John"", ""lastName"": ""Smith"", ""children"": [] ,""preferredColors"":[""Blue"", ""Black""]}"
Append to array,"{ ""firstName"": ""John"", ""lastName"": ""Smith"", ""children"": [""Annette""] }"
Replace an array with a scalar,"{ ""firstName"": ""John"", ""lastName"": ""Smith"", ""children"": ""Annette"" }"
Add an object,"{ ""firstName"": ""John"", ""lastName"": ""Smith"", ""children"": [] ,""phoneNumbers"":{""type"": ""home"",""number"": ""212 555-1234""}}"
Remove an object,"{ ""lastName"": ""Smith"", ""children"": [] }"


Changes can be done right in-place, without the need to load JSON into a variable. Here's an example:

In [10]:
-- Add a new sample user
insert into dbo.users_json
select 42, json_data = N'{"firstName": "Joe", "lastName": "Black"}';

-- Update last name
update 
	dbo.users_json
set
	json_data = json_modify(json_data, '$.lastName', 'Green')
where 
    id = 42;

-- Add Phone Number
update 
	dbo.users_json
set
	json_data = json_modify(	
            json_data,		
			'append $.phoneNumbers',
			json_query('{"type": "fax","number": "212 555-1234"}')
			)
where 
	json_value(json_data, '$.firstName') = 'Joe';

-- View result
select * from dbo.users_json where id = 42


(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.018

id,json_data
42,"{""firstName"": ""Joe"", ""lastName"": ""Green"",""phoneNumbers"":[{""type"": ""fax"",""number"": ""212 555-1234""}]}"


Clean up

In [9]:
delete from dbo.users_json where id = 42

(0 rows affected)

Total execution time: 00:00:00.007